In [ ]:
# import des librairy

import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt

#imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.linear_model import BayesianRidge

pd.options.display.max_columns = 999

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


### let's focus on application_train.csv

In [ ]:
#open the dataframe

df_application = pd.read_csv(r'C:\\Users\\alojou\\Notebook Python\\Projet_7 Modèle de scoring\\input\\df_train.csv')

## Format Data

In [ ]:
"""category_features = []
threshold = 10
for each in df_application.columns:
    if df_application[each].nunique() < threshold:
        category_features.append(each)

print("transformed from numerical to categorical : ",category_features)   
        
        
for each in category_features:
    df_application[each] = df_application[each].astype('category')"""

In [ ]:
"""obj_col = df_application.select_dtypes("object").columns
for each in obj_col:
    df_application[each] = df_application[each].astype('category')"""

In [ ]:
"""# split dtypes for futur analysis
numeric_col = df_application.select_dtypes(np.number).columns
cat_col = df_application.select_dtypes("category").columns"""

In [ ]:
"""print(numeric_col)
print(cat_col)"""

In [ ]:
df_application.info()

In [ ]:
df_application.head()

## Gestion des NAN

In [ ]:
df_application.isna().sum()[df_application.isna().sum()> 0].to_frame().T

In [ ]:
nan_col = df_application.isna().sum()[df_application.isna().sum()> 0].to_frame().T.columns

num_nan_col = df_application[nan_col].select_dtypes(np.number).columns
cat_nan_col = df_application[nan_col].select_dtypes("category").columns

In [ ]:
num_nan_col

## Imputation des nan 

## Numérique

In [ ]:
df_application.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
imputer = IterativeImputer(estimator = BayesianRidge(), n_nearest_features=5, imputation_order='ascending',skip_complete = True)

In [ ]:
# fit on the dataset
#imputer.fit(df_application[num_nan_col])
# transform the dataset
Xtrans = imputer.transform(df_application[num_nan_col])


In [ ]:
pd.DataFrame(Xtrans).info()

In [ ]:
trans_pd = pd.DataFrame(Xtrans).set_axis(num_nan_col, axis=1, inplace=False)

In [ ]:
df_application = df_application.drop(columns = num_nan_col)

In [ ]:
df_merged = df_application.join(trans_pd, how="inner" )

In [ ]:
df_merged

In [ ]:
df_merged.to_csv(r'C:\\Users\\alojou\\Notebook Python\\Projet_7 Modèle de scoring\\input\\df_train.csv')

In [ ]:
df_application.isna().sum()[df_application.isna().sum()> 0].to_frame().T

In [ ]:
df_application = df_merged

In [ ]:
df_application.info()

### Univariate analyis

In [ ]:
for colu in cat_col:
    plt.figure(figsize=(9, 5))
    ax = sns.countplot(y=colu, hue="TARGET",data=df_application)
    plt.show()

In [ ]:
## let's take a closer look on organization_type

df_application.ORGANIZATION_TYPE = df_application.ORGANIZATION_TYPE.apply(lambda x : str(x).split(" ")[0].replace(":",""))

In [ ]:
for colu in numeric_col:
    
    g = sns.FacetGrid(df_application, hue="TARGET", palette="Set1",height=4, aspect=2)
    g.map(sns.kdeplot, colu)
    plt.show()
    

## Multivariate analysis 

#### a revoir pd.scatter_matrix pour distinguer les relation linéaire et non linéaire

In [ ]:
#pairplot
#pd.plotting.scatter_matrix(df_application[numeric_col],hist_kwds={'bins':30})

In [ ]:
#corr_df_p = df_application.corr(method='pearson')
corr_df_s = df_application.corr(method='spearman')

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(corr_df_p[corr_df_p>0.8], annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(corr_df_s[corr_df_s>0.8], annot=True)
plt.show()

### drop correlated columns

In [ ]:
import numpy as np

# Create correlation matrix
corr_matrix = df_application.corr(method='spearman').abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.8
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

print("columns removed : ",to_drop)

# Drop features 
df_application.drop(to_drop, axis=1, inplace=True)

In [ ]:
#update numeric column 
numeric_col = [col for col in numeric_col if col not in to_drop]

## Statistical test 

In [ ]:
df_application.info()

### Boxplot

In [ ]:
sns.set_style("whitegrid")

for col in df_application.select_dtypes(np.number).columns : 
    plt.figure(figsize=(10,10))
    sns.boxplot(x = 'TARGET', y = col, data = df_application) 
    plt.show()
    pass

### Normal Test

In [ ]:
from scipy import stats

stats_normal_results = {}

for var in df_application.select_dtypes(np.number).columns:
    print("variable à tester :",var )
    
    k2, p = stats.normaltest(df_application[[var]])
    alpha = 0.05
    print("statistics = ",k2,"p = {}".format(p))
    
    if p < alpha:  # null hypothesis: x comes from a normal distribution
        print("The null hypothesis can be rejected\n")
        stats_normal_results[var] = "no normal"
    else:
        print("The null hypothesis cannot be rejected\n")
        stats_normal_results[var] = "normal"

In [ ]:
stats_normal_results

### mann_whitney_u_test

In [ ]:
df_target = df_application[df_application.TARGET==1]

In [ ]:
df_target

In [ ]:
df_sampled = df_application[df_application.TARGET==0].sample(frac =0.1)

In [ ]:
df_sampled

In [ ]:
def mann_whitney_u_test(distribution_1, distribution_2):
    """
    Perform the Mann-Whitney U Test, comparing two different distributions.
    Args:
       distribution_1: List. 
       distribution_2: List.
    Outputs:
        u_statistic: Float. U statisitic for the test.
        p_value: Float.
    """
    u_statistic, p_value = stats.mannwhitneyu(distribution_1, distribution_2)
    return u_statistic, p_value


In [ ]:
#Perform the Mann-Whitney U Test on the two distributions

stats_normal_results_2 ={}

for var in df_application.select_dtypes(np.number).columns:
    print("variable à tester :",var )

    u_statistic, p_value = mann_whitney_u_test(list(df_target[var]), list(df_sampled[var]))

    alpha = 0.05
    print("statistics = ",u_statistic,"p = {}".format(p))

    if p < alpha:  # null hypothesis: population differe
        print("The null hypothesis can be rejected\n")
        stats_normal_results_2[var] = "different"
    else:
        print("The null hypothesis cannot be rejected\n")
        stats_normal_results_2[var] = "same"

In [ ]:
stats_normal_results_2

In [ ]:
df_application.to_csv(r'C:\\Users\\alojou\\Notebook Python\\Projet_7 Modèle de scoring\\Data\\df_clean.csv')

In [ ]:
### next model